In [6]:
# Creator: Kathryn Zamiela

In [7]:
# Input: path to FASTA file
# Output: DNA sequence matching the file contents
def read_genome(filename):
    genome = ''

    with open(filename, 'r') as f:
        for line in f:
            # want to ignore >
            if not line[0] == '>':
                genome += line.rstrip() # removes trailing whitespace from end of string

    return genome


# Input: path to FASTQ file
# Output: one list containing the DNA sequence and one list containing base quality values for the DNA sequence
def read_fastq(filename):
    sequences = []
    qualities = []

    with open(filename) as f:
        while True:
            f.readline() # ignoring first line
            seq = f.readline().rstrip() # save sequence line
            f.readline() # ignoring third line
            qual = f.readline().rstrip() # save base qualities

            # check if at end of file
            if len(seq) == 0:
                break

            # save to lists
            sequences.append(seq)
            qualities.append(qual)

    return sequences, qualities

In [8]:
# Input: string containing characters 'ACGTN'
# Output: matching DNA base pairs in the reverse order
def reverse_complement(s):
    # create complement dictionary
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}

    t = ''

    for base in s:
        t = complement[base] + t # prepending reverses the string

    return t

In [ ]:
# Input: string of pattern to match, string of text to match, int of position in text
# Output: boolean indicating match status
def is_match(pattern, text, i):
    match = True

    # check each character in the pattern based on your position in the text
    for j in range(len(pattern)):
     # if the characters don't match, it's not a match
        if not text[i + j] == pattern[j]:
            match = False
            break

    return match


# Input: string of pattern to match, string of text to match
# Output: a list of indices where the pattern matches the text
#         strand-aware: will check for a match of the reverse complement also
# Specifications: will only report a case where pattern and reverse complement match each other once
def naive(pattern, text):
    occurrences = [] # all indices where p matches t

    # iterate through the text without going over
    for i in range(len(text) - len(pattern) + 1):
        # check if the pattern matches
        match_f = is_match(pattern, text, i)

        # get reverse complement of pattern
        rev_pattern = reverse_complement(pattern)

        # check if reverse complement matches
        match_r = is_match(rev_pattern, text, i)
        
        # if the original pattern and reverse complement are the same, using the or statement will only save it once
        if match_f == True or match_r == True:
            occurrences.append(i)

    return occurrences

In [18]:
# Input: string of pattern to match, string of text to match, int of position in text
# Output: boolean indicating match status
# Specifications: Allows for up to 2 mismatches per occurrence
def is_approx_match(pattern, text, i):
    match = True

    # set error counter
    err_count = 0

    # check each character in the pattern based on your position in the text
    for j in range(len(pattern)):
     # if the characters don't match
        if not text[i + j] == pattern[j]:
            # increment error counter
            err_count += 1

            # if there have been more than 2 mismatches, not a match
            if err_count > 2:
                match = False
                break

    return match
    

# Input: string of pattern to match, string of text to match
# Output: a list of indices where the pattern matches the text
# Specifications: Allows for up to 2 mismatches per occurrence
#                 Does not consider reverse complement
def naive_2mm(pattern, text):
    occurrences = [] # all indices where p matches t

    # iterate through the text without going over
    for i in range(len(text) - len(pattern) + 1):
        # check if the pattern matches
        match = is_approx_match(pattern, text, i)
        
        # if the original pattern and reverse complement are the same, using the or statement will only save it once
        if match == True:
            occurrences.append(i)

    return occurrences

In [42]:
# Sample usage of this block: 
# seqs, quals = read_fastq(filename)
# worst_idx = find_worst_seq_cycle(quals)
# print(worst_idx)


# Input: Phred+33 int value
# Output: character representing base quality from FASTQ file
def q_to_phred33(q):
    return chr(q + 33) # chr converts char to int according to ASCII


# Input: character representing base quality from FASTQ file
# Output: Phred+33 int value
def phred33_to_q(qual):
    return ord(qual)-33 # ord converts int to char according to ASCII


# Input: list of base qualities from FASTQ file
# Output: list of average qualities, index corresponds to FASTQ read index
# Specifications: assumes that all reads are of the same length
def create_seq_cycle_avgs(quals):
    seq_cycle_avg = []

    # calculate length of read
    len_read = len(quals[0])

    # iterate through each base
    for i in range(len_read):
        # initialize total
        total_qual = 0

        # iterate through each read
        for qual in quals:
            # get int value
            val = phred33_to_q(qual[i])

            # add to total
            total_qual += val

        # calculate average
        seq_cycle_avg.append(total_qual / float(len_read))

    # return list of averages
    return seq_cycle_avg


# Input: list of sequencing cycle averages from FASTQ data
# Output: a float of the lowest average found as well as index found at
def find_lowest_val_with_idx(l):
    lowest_val = 0.0
    lowest_idx = -1

    for i in range(len(l)):
        # if lowest val hasn't been set, use current val
        if lowest_idx == -1:
            lowest_val = l[i]
            lowest_idx = i
        elif l[i] < lowest_val:
            lowest_val = l[i]
            lowest_idx = i

    return lowest_val, lowest_idx


# Input: list of base qualities from FASTQ file
# Output: index of poorest sequencing cycle
# Specifications: assumes that all reads are of the same length
def find_worst_seq_cycle(quals):
    # create list of cycle averages
    seq_cycle_avg = create_seq_cycle_avgs(quals)

    # find lowest average
    lv, li = find_lowest_val_with_idx(seq_cycle_avg)
    
    return li